In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split 

In [2]:
from matplotlib.lines import Line2D
import seaborn as sns

import numpyro
import numpyro.distributions as dist
from numpyro import handlers
from numpyro.infer import MCMC, NUTS

import jax
import jax.numpy as jnp
from jax import random, vmap
from jax.scipy.special import logsumexp
from jax import lax

In [3]:
DATA_PATH = '../data/processed/data_processed.csv'
data = pd.read_csv(DATA_PATH)

In [4]:
data.shape

(290642, 42)

In [5]:
data.columns

Index(['Unnamed: 0', 'machineID', 'datetime', 'voltmean_3h', 'rotatemean_3h',
       'pressuremean_3h', 'vibrationmean_3h', 'voltsd_3h', 'rotatesd_3h',
       'pressuresd_3h', 'vibrationsd_3h', 'voltmean_24h', 'rotatemean_24h',
       'pressuremean_24h', 'vibrationmean_24h', 'voltsd_24h', 'rotatesd_24h',
       'pressuresd_24h', 'vibrationsd_24h', 'error1count', 'error2count',
       'error3count', 'error4count', 'error5count', 'comp1', 'comp2', 'comp3',
       'comp4', 'comp1_life', 'comp2_life', 'comp3_life', 'comp4_life', 'age',
       'model_model1', 'model_model2', 'model_model3', 'model_model4',
       'failure', 'comp1_fail', 'comp2_fail', 'comp3_fail', 'comp4_fail'],
      dtype='object')

In [6]:
data['datetime'] = pd.to_datetime(data.datetime)

In [7]:
data_per_id = data.loc[data.machineID==1]

In [8]:
data_cols = ['voltmean_3h', 'rotatemean_3h',
       'pressuremean_3h', 'vibrationmean_3h', 'voltsd_3h', 'rotatesd_3h',
       'pressuresd_3h', 'vibrationsd_3h', 'voltmean_24h', 'rotatemean_24h',
       'pressuremean_24h', 'vibrationmean_24h', 'voltsd_24h', 'rotatesd_24h',
       'pressuresd_24h', 'vibrationsd_24h', 'error1count', 'error2count',
       'error3count', 'error4count', 'error5count','model_model1', 'model_model2', 'model_model3', 'model_model4']

target_cols = ['comp1_life', 'comp2_life', 'comp3_life', 'comp4_life']

In [9]:
N_features = len(data_cols)

In [10]:
X_df = data_per_id[data_cols]
X = X_df.values

y_df = [data_per_id[target_col] for target_col in target_cols]

for df_ in y_df:
    df_.dropna(inplace=True, axis=0)

# y = np.array([y_df_i.values for y_df_i in y_df]).T

/tmp/ipykernel_18106/2464161881.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_.dropna(inplace=True, axis=0)


In [11]:
a = data_per_id['comp1_life']

In [12]:
a = a.dropna()

In [13]:
y = a.values

In [14]:
train_size_pc = 0.85
train_idx = np.floor(train_size_pc*X.shape[0]).astype(int)

X_train = X[:train_idx]
X_test = X[train_idx:]

y_train = y[:train_idx]
y_test = y[train_idx:]

In [15]:
N_train = y_train.shape[0]
N_test = y_test.shape[0]

In [59]:
def f(carry, inputs):
    
    
#     noise_t, x_prev = inputs

    inpts = inputs
    
    
    beta, z_prev, tau, R = carry
    
    p_u = R.shape[0]
        
    # Create B matrix
    k = -1
    ones_ = jnp.ones(beta.shape[0]-np.abs(k))
    B = jnp.diag(ones_, k=k)
    B = B.at[0].set(beta)
    
    # Create R_u matrix
    R_u = jnp.zeros((beta.shape[0],inpts.shape[0]))
    print(R_u.shape)
    print(R.shape)
    
#     R_u = R_u.at[0].set(R)
    
    z_t = jnp.dot(B, z_prev) + jnp.dot(R, inpts) # + noise_t
        
    z_prev = z_t
        
    return (beta, z_prev, tau, R), z_t

In [62]:
def model(T, T_forecast, n_states=4, obs=None, inputs=None, n_previous=1):
    """
    Define priors over beta, tau, sigma, z_1 (keep the shapes in mind)
    """
    beta = numpyro.sample(name="beta", fn=dist.Normal(loc=jnp.zeros(n_states), scale=jnp.ones(n_states)))
    tau = numpyro.sample(name="tau", fn=dist.HalfCauchy(scale=jnp.ones(n_states)))
    sigma = numpyro.sample(name="sigma", fn=dist.HalfCauchy(scale=.1))
    z_prev = numpyro.sample(name="z_1", fn=dist.Normal(loc=jnp.zeros(n_states), scale=jnp.ones(n_states)))
    
    n,p = inputs.shape
    
    
    R = numpyro.sample(name="R", fn=dist.Normal(loc=jnp.zeros((n_states, p)), scale=jnp.ones((n_states, p))))
    
    
    """
    Define LKJ prior
    """
    L_Omega = numpyro.sample("L_Omega", dist.LKJCholesky(n_states, 10.))
    Sigma_lower = jnp.matmul(jnp.diag(jnp.sqrt(tau)), L_Omega) # lower cholesky factor of the covariance matrix
#     noises = numpyro.sample("noises", fn=dist.MultivariateNormal(loc=jnp.zeros(n_states), scale_tril=Sigma_lower), sample_shape=(T+T_forecast-n_previous,))
        
    

    
    """
    Propagate the dynamics forward using jax.lax.scan
    """
    carry = (beta, z_prev, tau, R)
    z_collection = [z_prev]
    
    
    carry, zs_exp = lax.scan(f, carry, inputs, T+T_forecast)
        
    z_collection = jnp.concatenate((jnp.array(z_collection), zs_exp), axis=0)
    
    print(z_collection[:T, 0].shape)
    
    numpyro.sample(name="y_obs1", fn=dist.Normal(loc=z_collection[:T, 0], scale=sigma), obs=obs)
    numpyro.sample(name="y_pred1", fn=dist.Normal(loc=z_collection[T:, 0], scale=sigma), obs=None)    

    return z_collection

In [63]:
rng_key = random.PRNGKey(0)
rng_key, rng_key_ = random.split(rng_key)

n_states = 14
n_previous = 1
T = N_train
T_forecast = N_test


X_t = np.concatenate([X_train[n_previous:,:], X_test], axis=0)


nuts_kernel = NUTS(model=model, max_tree_depth=8, step_size=5e-3, adapt_step_size=False)
mcmc = MCMC(nuts_kernel, num_samples=3000, num_warmup=1000, num_chains=1)
mcmc.run(rng_key_, T=T, T_forecast=T_forecast, n_states=n_states, inputs=X_t, obs=y_train, n_previous=n_previous)

(2476,)
(2476,)
(2476,)
(2476,)


ValueError: Normal distribution got invalid loc parameter.

In [ ]:
y_train.shape

In [ ]:
mcmc.print_summary()

In [ ]:
samples_ = mcmc.get_samples()

In [ ]:
plt.plot(samples_['y_pred1'].mean(axis=0))
plt.plot(y_test[:,0])
plt.show()